<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-28 10:51:39
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.34 C
-------------------
Today PnL: -12.08 K (-0.09%)
Current PnL: -17.04 L (-11.87%)
CY Booked + Current PnL: -8.96 L (-6.24%)
-------------------
Total profit:  2.80 L
Total loss:  -19.85 L
-------------------
Total Booked + Current PnL: 18.02 L (15.46%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.02%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 75.25 L (56.11%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.1%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
92      VIPIND   488.80   489.00  -218.93       H-SC    11.47   85820.0   
39    HINDALCO   651.95   761.55   -20.73       H-LC     5.40  109856.0   
4   APOLLOHOSP  6904.43  8285.00   -18.07       H-LC     6.84  111682.0   
58     MEDANTA  1087.93  1486.00    24.39       H-SC    10.64  159488.0   
17   BRITANNIA  5190.70  6446.05    13.41       H-LC     9.77  101700.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
92      -9496.0   9535.0       -2.06         -9.96  11.11   0.04    161.0   
39       5544.0  11996.0       -0.99          5.31  10.92  16.81     10.0   
4        8116.0  12598.0       -0.31          7.84  11.28  20.00     14.0   
58      33288.0  12887.0        3.91         26.38   8.08  36.59    154.0   
17       8267.0  14330.0        1.00          8.85  14.09  24.18     35.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
92    -1.00        0.64    69.27    OX40N      NTT        MISC  
39     0.46        0.82    22.21      X5K      ATH      METALS  
4      0.64        0.83    23.01     X40N      BTT  HEALTHCARE  
58     2.58        1.19    42.20     XY24      NTT  HEALTHCARE  
17     0.58        0.76    23.49     XY25      ATH        FMCG

In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
14  BERGEPAINT   561.33   680.00   -18.91       H-MC    11.21  229534.0   
38  HEROMOTOCO  4311.81  5949.07    -6.67       H-MC     2.82  149790.0   
59  NATIONALUM   189.63   247.44   -50.15       H-MC     3.64  100196.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
14       2195.0  45861.0        1.57          0.97  19.98  21.14    108.0   
38      -1123.0  58433.0        1.16         -0.74  39.01  37.97     89.0   
59       -308.0  30951.0       -2.38         -0.31  30.89  30.49     81.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
14     0.05        1.71    28.51     XY24      NTT   PAINTS  
38    -0.02        1.12    24.37       AR      ATH     AUTO  
59    -0.01        0.75    32.00       MH      ATH   METALS

In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction  Spread%   Current  \
22       CIPLA  1492.70  1795.0   -18.76       H-LC     1.07  213926.0   
56        LTIM  5564.16  7230.2    -4.21       H-LC     2.41  184122.0   
10    BAJAJHFL   122.26   152.0    -7.95       H-LC     2.46  191011.0   
40  HINDUNILVR  2413.81  2723.0   -12.66       H-LC     3.01  273493.0   
5   ASIANPAINT  2961.56  4250.0   -19.51       H-LC     3.53  159922.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
22       9426.0   31982.0        1.89          4.61  14.95  20.25      7.0   
56     -16188.0   76171.0        0.36         -8.08  41.37  29.94     26.0   
10      -9495.0   58277.0       -0.47         -4.74  30.51  24.33     30.0   
40       3146.0   31479.0        1.10          1.16  11.51  12.81     18.0   
5      -41464.0  129073.0        0.67        -20.59  80.71  43.51     20.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
22     0.29        1.60    13.74     X40N      BTT   PHARMA  
56    -0.21        1.37    27.20     X200      ATH       IT  
10    -0.16        1.43    10.15     X40N      BTT  FINANCE  
40     0.10        2.04    14.38     XY25      NTT     FMCG  
5     -0.32        1.19    11.32      X40      BTT   PAINTS

In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
58     MEDANTA  1087.93  1486.00    24.39       H-SC    10.64  159488.0   
72      SHALBY   261.39   327.00   979.74       M-SC     9.72  146847.0   
22       CIPLA  1492.70  1795.00   -18.76       H-LC     1.07  213926.0   
91         VBL   492.64   672.28   -13.77       H-LC    12.72  261549.0   
14  BERGEPAINT   561.33   680.00   -18.91       H-MC    11.21  229534.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
58      33288.0   12887.0        3.91         26.38   8.08  36.59    154.0   
72     -34296.0   79767.0        2.10        -18.93  54.32  25.10    228.0   
22       9426.0   31982.0        1.89          4.61  14.95  20.25      7.0   
91      -3491.0  100147.0        1.89         -1.32  38.29  36.46      2.0   
14       2195.0   45861.0        1.57          0.97  19.98  21.14    108.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
58     2.58        1.19    42.20     XY24      NTT  HEALTHCARE  
72    -0.43        1.10    17.68     XY24      NTT  HEALTHCARE  
22     0.29        1.60    13.74     X40N      BTT      PHARMA  
91    -0.03        1.95    11.70     X40N      ATH        FMCG  
14     0.05        1.71    28.51     XY24      NTT      PAINTS

In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
67        SAIL   130.64   228.00    79.57       M-MC    13.81  215594.0   
16   BLUSPRING   226.45    94.95      NaN        NaN     5.61   19841.0   
59  NATIONALUM   189.63   247.44   -50.15       H-MC     3.64  100196.0   
2          ACC  2579.05  3906.00   -37.64       M-MC     4.85  134882.0   
92      VIPIND   488.80   489.00  -218.93       H-SC    11.47   85820.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
67      -9368.0  177024.0       -4.19         -4.16   82.11  74.53    193.0   
16     -29752.0     952.0       -3.30        -59.99    4.80 -58.07    204.0   
59       -308.0   30951.0       -2.38         -0.31   30.89  30.49     81.0   
2      -53389.0  150259.0       -2.24        -28.36  111.40  51.45    175.0   
92      -9496.0    9535.0       -2.06         -9.96   11.11   0.04    161.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
67    -0.05        1.61    25.17     XY24      BTT    STEEL  
16   -31.25        0.15    33.06     XY24      ATH     MISC  
59    -0.01        0.75    32.00       MH      ATH   METALS  
2     -0.36        1.01     2.55     XY24      BTT   CEMENT  
92    -1.00        0.64    69.27    OX40N      NTT     MISC

In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction  Spread%   Current  \
92      VIPIND   488.80   489.0  -218.93       H-SC    11.47   85820.0   
78    SYMPHONY  1306.42  1306.0   -30.70       M-SC     9.75  147506.0   
86  UJJIVANSFB    52.77    53.0    35.36       M-SC    22.43  117909.0   
80   TATAELXSI  7332.28  7332.0   -19.33       H-MC     1.60   84749.0   
21        CERA  7989.07  8422.0   -17.42       H-SC     3.08  117936.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%  OTT%  CumlRnk  \
92      -9496.0   9535.0       -2.06         -9.96  11.11  0.04    161.0   
78     -23635.0  23586.0        0.90        -13.81  15.99 -0.03    194.0   
86     -24570.0  25185.0        0.81        -17.24  21.36  0.44    249.0   
80     -17903.0  17899.0       -0.14        -17.44  21.12 -0.00    100.0   
21     -25867.0  33659.0       -0.93        -17.99  28.54  5.42    155.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
92    -1.00        0.64    69.27    OX40N      NTT      MISC  
78    -1.00        1.10     6.74    OX40N      NTT  DURABLES  
86    -0.98        0.88    39.17    OX40N      NTT     BANKS  
80    -1.00        0.63    28.83    OX40N      NTT        IT  
21    -0.77        0.88    30.15    OX40N      NTT  CERAMICS

In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
32  ENRIN  1377.95  3176.70      NaN        NaN     2.49   75002.0   
91    VBL   492.64   672.28   -13.77       H-LC    12.72  261549.0   
82    TCS  3794.03  4998.00   -25.77       H-LC    12.37  242073.0   
48   INFY  1461.46  2275.00   -16.01       H-LC     6.89  269502.0   
1     ABB  5551.76  7934.00   -38.31       H-LC     7.05  220077.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
32      40553.0    4418.0        0.76        117.72   5.89  130.54      1.0   
91      -3491.0  100147.0        1.89         -1.32  38.29   36.46      2.0   
82     -53861.0  147761.0       -1.03        -18.20  61.04   31.73      3.0   
48       7901.0  137716.0       -0.67          3.02  51.10   55.67      5.0   
1        3558.0   89351.0       -0.98          1.64  40.60   42.91      6.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
32     9.18        0.56    14.69       AR      ATH  ELECTRICAL  
91    -0.03        1.95    11.70     X40N      ATH        FMCG  
82    -0.36        1.81     0.00      X40      BTT          IT  
48     0.06        2.01     9.31      X40      BTT          IT  
1      0.04        1.64    15.04       AR      NTT  ELECTRICAL

In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
28  DIGITIDE   188.38   267.99      NaN        NaN     2.36   53832.0   
58   MEDANTA  1087.93  1486.00    24.39       H-SC    10.64  159488.0   
32     ENRIN  1377.95  3176.70      NaN        NaN     2.49   75002.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%  FTT%    OTT%  CumlRnk  \
28      12577.0   4856.0       -1.77         30.49  9.02   42.26    197.0   
58      33288.0  12887.0        3.91         26.38  8.08   36.59    154.0   
32      40553.0   4418.0        0.76        117.72  5.89  130.54      1.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
28     2.59        0.40    26.74     XY24      ATH          IT  
58     2.58        1.19    42.20     XY24      NTT  HEALTHCARE  
32     9.18        0.56    14.69       AR      ATH  ELECTRICAL

In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
53    KPIGREEN   497.21   731.64    25.90       H-SC    12.73  128709.0   
58     MEDANTA  1087.93  1486.00    24.39       H-SC    10.64  159488.0   
20        CAMS  3643.00  5226.82    -1.02       H-SC     1.18  111135.0   
15  BLUESTARCO  1646.70  2326.38    -6.81       H-SC    10.50  174800.0   
26       DABUR   505.20   735.00    -6.82       H-MC     4.27  201962.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53       3412.0  55667.0       -1.75          2.72  43.25  47.15    143.0   
58      33288.0  12887.0        3.91         26.38   8.08  36.59    154.0   
20       9131.0  35219.0       -1.62          8.95  31.69  43.48    122.0   
15      10130.0  57841.0        0.05          6.15  33.09  41.28    119.0   
26       4934.0  84683.0        1.27          2.50  41.93  45.49     96.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53     0.06        0.96    61.02       MH      ATH       POWER  
58     2.58        1.19    42.20     XY24      NTT  HEALTHCARE  
20     0.26        0.83    27.23     X40N      ATH        MISC  
15     0.18        1.30    14.63     X40N      ATH          AC  
26     0.06        1.51    15.87     XY24      BTT        FMCG

In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
32      ENRIN  1377.95  3176.70      NaN        NaN     2.49   75002.0   
94  WHIRLPOOL  1219.98  2270.00   -38.94       M-SC     8.85  102765.0   
45  INDIAMART  2327.09  4850.92   -47.70       H-SC    12.86  136252.0   
53   KPIGREEN   497.21   731.64    25.90       H-SC    12.73  128709.0   
95      WIPRO   243.46   420.00    -8.76       M-LC     5.67  156178.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
32      40553.0    4418.0        0.76        117.72   5.89  130.54      1.0   
94      11266.0   67486.0       -2.06         12.31  65.67   86.07    205.0   
45      12916.0  120842.0        0.40         10.47  88.69  108.45    118.0   
53       3412.0   55667.0       -1.75          2.72  43.25   47.15    143.0   
95       5233.0  104218.0       -0.90          3.47  66.73   72.51     56.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
32     9.18        0.56    14.69       AR      ATH  ELECTRICAL  
94     0.17        0.77    48.66       XR      NTT    DURABLES  
45     0.11        1.02    35.17       AR      ATH        MISC  
53     0.06        0.96    61.02       MH      ATH       POWER  
95     0.05        1.17     9.70       XR      NTT          IT

In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction  Spread%  Current  \
16  BLUSPRING   226.45   94.95      NaN        NaN     5.61  19841.0   
28   DIGITIDE   188.38  267.99      NaN        NaN     2.36  53832.0   
74        SIS   477.00  477.00  2293.78       M-SC     5.77  64396.0   
24  COFFEEDAY    59.14   80.00   -53.72       L-SC    20.97  68640.0   
92     VIPIND   488.80  489.00  -218.93       H-SC    11.47  85820.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
16     -29752.0    952.0       -3.30        -59.99    4.80 -58.07    204.0   
28      12577.0   4856.0       -1.77         30.49    9.02  42.26    197.0   
74     -16694.0  16691.0       -0.33        -20.59   25.92   0.00    238.0   
24     -44909.0  84963.0       -0.83        -39.55  123.78  35.27    267.0   
92      -9496.0   9535.0       -2.06         -9.96   11.11   0.04    161.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
16   -31.25        0.15    33.06     XY24      ATH     MISC  
28     2.59        0.40    26.74     XY24      ATH       IT  
74    -1.00        0.48    30.51    OX40N      NTT     MISC  
24    -0.53        0.51    68.00       XR      NTT   HOTELS  
92    -1.00        0.64    69.27    OX40N      NTT     MISC

In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction  Spread%   Current  \
28    DIGITIDE   188.38   267.99      NaN        NaN     2.36   53832.0   
58     MEDANTA  1087.93  1486.00    24.39       H-SC    10.64  159488.0   
33  GREENPANEL   375.16   537.00   250.78       M-SC     9.88  156575.0   
16   BLUSPRING   226.45    94.95      NaN        NaN     5.61   19841.0   
72      SHALBY   261.39   327.00   979.74       M-SC     9.72  146847.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
28      12577.0    4856.0       -1.77         30.49   9.02  42.26    197.0   
58      33288.0   12887.0        3.91         26.38   8.08  36.59    154.0   
33     -26503.0  105485.0        0.11        -14.48  67.37  43.14    234.0   
16     -29752.0     952.0       -3.30        -59.99   4.80 -58.07    204.0   
72     -34296.0   79767.0        2.10        -18.93  54.32  25.10    228.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
28     2.59        0.40    26.74     XY24      ATH          IT  
58     2.58        1.19    42.20     XY24      NTT  HEALTHCARE  
33    -0.25        1.17    47.28     XY24      NTT        MISC  
16   -31.25        0.15    33.06     XY24      ATH        MISC  
72    -0.43        1.10    17.68     XY24      NTT  HEALTHCARE

In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       18.47
1     25       41.16
2     50       70.18

In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.74
LC    35.13
MC    22.04
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.93
X40N     14.60
X40      12.50
XY25     10.78
XR        9.16
OX40N     8.34
AR        8.25
X5K       2.24
MH        1.71
X200      1.37
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.86
H-SC    25.32
H-MC    19.07
M-SC    15.01
M-LC     7.21
M-MC     2.62
L-SC     1.41
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                14.12         -3.43   38.04
IT                  11.75        -15.37   72.21
FINANCE              8.36        -16.33   58.54
BANKS                7.58        -12.27   58.08
MISC                 6.06        -18.50   73.84
PAINTS               5.59        -16.38   39.31
ELECTRICAL           5.43         -1.21   49.17
HEALTHCARE           4.67          3.52   22.10
AUTO                 4.42        -14.28   64.01
AC                   3.49         -3.18   38.12
INSURANCE            3.26         -8.22   42.58
METALS               2.99         -2.35   42.49
DURABLES             2.49         -9.16   32.83
FOOTWEAR             2.39        -28.83   71.40
TRAVEL               2.19        -35.82  100.11
CERAMICS             1.97        -27.98   69.67
STEEL                1.61         -4.35   82.11
PHARMA               1.60          4.41   14.95
CHEMICALS            1.43        -21.72  114.80
BREWERIES            1.30         -4.34   24.57
ENGINEERING          1.16        -23.60   74.37
JEWELLERY            1.15        -87.72  135.89
MINING               1.06         -6.31   33.90
CEMENT               1.01        -39.58  111.40
POWER                0.96          2.65   43.25
ENTERTAINMENT        0.90        -35.81  104.09
TEXTILES             0.57        -20.17   54.44
HOTELS               0.51        -65.43  123.78

In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      2708507.0     25
XR        1065626.0     13
AR        1039185.0     10
X40        777876.0     10
X40N       594090.0     12
XY25       494197.0      8
OX40N      354508.0     11
SR         189494.0      2
X5K        139216.0      2
MH          86618.0      2
X200        76171.0      1

In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2424316.0     26
M-SC        1618965.0     20
H-LC        1264766.0     20
H-MC        1151428.0     15
M-LC         414334.0      5
M-MC         327283.0      2
L-SC         210226.0      3
L-MC          55967.0      1
L-LC          47977.0      1

In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      938129.0      7
M-SC       XY24      773470.0      7
H-SC       AR        518538.0      3
H-LC       X40       504977.0      6
H-MC       XY24      444321.0      4
H-SC       XR        352593.0      4
M-MC       XY24      327283.0      2
H-LC       X40N      305531.0      6
M-SC       XR        285802.0      4
H-SC       X40N      214578.0      4
H-LC       AR        212053.0      2
H-MC       XY25      209901.0      2
           X40       198817.0      3
H-SC       SR        189494.0      2
M-SC       AR        185438.0      2
           OX40N     181292.0      5
M-LC       XY24      167099.0      2
H-SC       OX40N     155317.0      5
L-SC       XR        149921.0      2
M-LC       X5K       127220.0      1
M-SC       XY25      118881.0      1
H-MC       XR        117125.0      1
M-LC       XR        104218.0      1
H-LC       XY25      101641.0      3
           X200       76171.0      1
M-SC       X40        74082.0      1
H-MC       X40N       73981.0      2
L-SC       AR         60305.0      1
H-MC       AR         58433.0      1
L-MC       XR         55967.0      1
H-SC       MH         55667.0      1
H-LC       XY24       52397.0      1
L-LC       XY25       47977.0      1
H-MC       MH         30951.0      1
           OX40N      17899.0      1
M-LC       XY25       15797.0      1
H-LC       X5K        11996.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
